In [8]:
import re
import os
from glob import glob

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from operator import itemgetter
from langchain.schema import HumanMessage
from langchain_community.tools import TavilySearchResults
from langchain.vectorstores import Chroma

from textwrap import dedent
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_openai import ChatOpenAI


In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI Embeddings & LLM 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
chat_model = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


# 대화 메모리 설정
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# 벡터 스토어 로드 (이미 저장된 벡터 데이터 사용)
PERSIST_DIRECTORY = "vector_store/action"  # 기존에 데이터 저장된 경로
COLLECTION_NAME = "action_data"

vector_store = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model
)
context = ""  # 필요시 벡터스토어 검색 결과를 추가
history = []  # 처음 실행하는 경우 빈 리스트
# Retriever 설정 (유사한 웹툰 검색)
retriever = vector_store.as_retriever(search_type="mmr",search_kwargs={'k':10,'lambda_mult':0.25})


C:\Users\Playdata\AppData\Local\Temp\ipykernel_18648\3143691884.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18648\3143691884.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [ ]:
# db 검색 tool
@tool
def finder(user_query: str) -> list[Document]:
    """
    A tool which finds webtoons(or webnovel) using LLM
    """
    
    # 벡터스토어에서 검색
    search_results = retriever.invoke(user_query)
    
    # 검색 결과가 없을 경우 기본 메시지 제공
    if not search_results:
        return "관련된 웹툰 정보를 찾을 수 없습니다."

    # 검색된 Document 객체에서 텍스트 추출하여 context 구성
    context = "\n\n".join([doc.page_content for doc in search_results])

    # LLM 프롬프트 작성
    recommend_prompt = f"""
    <role>
    You are a webtoon/webnovel finder AI.
    find requested webtoons based on the given (context) data. 
    
    
    If no related webtoons are found, respond with "No recommendations available."
    
    사용자 입력: "{user_query}"
    
    (context)
    {context}
    </role>
    """

    # LLM 호출
    response = chat_model.invoke(recommend_prompt)
    
    return response.content.strip()


@tool
def classify_intent(user_query: str) -> str:
    """
    LLM을 사용하여 사용자의 의도, 감정, 말투를 분석하는 tool.
    """
    intent_prompt = f"""
    <basic role>
    사용자의 입력을 보고 의도와 감정, 말투를 분석하여 아래 중 하나로 분류하세요. 
    {{
    "가능한 의도": ["웹툰 추천 요청", "웹툰 정보 요청", "웹툰 인기 순위", "웹소설 추천 요청", "웹소설 정보 요청", "웹소설 인기 순위", "일반 대화", "인사"],
    "가능한 감정": ["평온", "기쁨", "슬픔", "화남", "기대", "장난"],
    "가능한 말투": ["반말", "존댓말"]
    }}
    </basic role>
    <rules>
    아니, 뭐해, 이딴, 아오 등은 사용자가 화났을 때 주로 사용.
    존댓말은 보통 "~요","~니다"로 끝남. 반말은 "~요","~니다"로 끝나지 않는 모든 말.
    {{
    examples of "존댓말": ["안녕하세요", "좋은 아침입니다", "확인했습니다", "금요일 웹툰 추천해주세요"],
    examples of "반말": ["안녕", "뭐해?", "어이없네", "금요일 웹툰 추천해줘"]
    }}
    "안녕"은 반말이야
    </rules>
    분석된 의도: , 분석된 감정: , 분석된 말투:

    사용자 입력: "{user_query}"
    """
    response = chat_model.invoke(intent_prompt)
    return response.content.strip()


@tool
def recommender(user_query: str) -> str:
    """
    A tool which recommends a list of webtoon(or webnovel) using LLM
    """
    
    # 벡터스토어에서 검색
    search_results = retriever.invoke(user_query)
    
    # 검색 결과가 없을 경우 기본 메시지 제공
    if not search_results:
        return "관련된 웹툰 정보를 찾을 수 없습니다."

    # 검색된 Document 객체에서 텍스트 추출하여 context 구성
    context = "\n\n".join([doc.page_content for doc in search_results])

    # LLM 프롬프트 작성
    recommend_prompt = f"""
    <role>
    You are a webtoon recommendation AI.
    Recommend 5 webtoons based on the given (context) data. 
    The recommended criterias are ratings, views, and like.
    Provide information in the following format:
    
    - Title:
    - Author:
    - Genre:
    - Description:
    - Platform:
    
    If no related webtoons are found, respond with "No recommendations available."
    
    사용자 입력: "{user_query}"
    
    (context)
    {context}
    </role>
    """

    # LLM 호출
    response = chat_model.invoke(recommend_prompt)
    
    return response.content.strip()

    

In [5]:
recommender("판타지 웹툰")

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21464\3852600451.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  recommender("판타지 웹툰")


'Here are five fantasy webtoons based on the criteria of ratings, views, and likes:\n\n1. **Title:** 언더클래스 히어로  \n   **Author:** 김우준  \n   **Genre:** 판타지  \n   **Description:** 도시 한가운데서 일어난 정체불명의 대폭발. 스승의 저주를 받아 천개의 선행을 하게 된 주인공 인(燐), 마을의 원수를 찾아 여행하는 검객 류진(劉進), 그리고 정체불명의 약초상 아미(俄靡). 대폭발의 원인을 찾아 떠나는 그들의 여행!  \n   **Platform:** 네이버 웹툰  \n\n2. **Title:** 바람이 머무는 난  \n   **Author:** 신월  \n   **Genre:** 판타지  \n   **Description:** 어느날 지상 최후의 용과 마주하게 된 한 여자아이의 이야기.  \n   **Platform:** 네이버 웹툰  \n\n3. **Title:** 쌍갑포차  \n   **Author:** 배혜수  \n   **Genre:** 판타지  \n   **Description:** 예상치 못한 시간, 예상치 못한 장소에 나타나는 쌍갑포차! 쌍갑포차를 운영하는 월주는 과연 누구일까? 감동적인 이야기들이 얽히는 힐링 웹툰.  \n   **Platform:** 카카오웹툰  \n\n4. **Title:** 체인소 맨  \n   **Author:** 후지모토 타츠키  \n   **Genre:** 액션  \n   **Description:** 악마 포치타와 함께 빚쟁이 데빌 헌터로 고용되어 혹사당하는 극빈곤 소년 덴지. 잔인한 배신을 계기로 악마가 깃든 몸으로 악마를 사냥하는 신세대 다크 히어로 액션, 개막!  \n   **Platform:** 카카오페이지  \n\n5. **Title:** 반선 [독점]  \n   **Author:** 약천수  \n   **Genre:** 무협  \n   **Description:** 요괴와 사람

In [ ]:
finder()

In [49]:
import uuid
from langchain.memory import ConversationBufferMemory

# 세션별 메모리를 저장할 글로벌 딕셔너리
session_memory = {}

def get_memory(session_id: str):
    """세션 ID별로 ConversationBufferMemory를 유지"""
    if session_id not in session_memory:
        session_memory[session_id] = ConversationBufferMemory(memory_key="history", return_messages=True)
    return session_memory[session_id]

def recommend_webtoons(query: str, session_id: str = None) -> str:
    # 세션 ID 자동 생성 (세션 ID가 제공되지 않은 경우)
    if session_id is None:
        session_id = uuid.uuid4().hex  # ✅ 자동 생성

    # 세션별 대화 메모리 가져오기
    memory = get_memory(session_id)

    # LLM 모델 설정
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 벡터스토어에서 검색한 결과를 context로 설정
    search_results = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in search_results]) if search_results else "관련된 웹툰 정보를 찾을 수 없습니다."

    # 대화 내역을 memory에서 불러오기
    history = memory.load_memory_variables({}).get("history", [])

    # 프롬프트 내부에서 context를 직접 포함하도록 변경
    prompt_template = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder("agent_scratchpad"),  # 🔹 추가된 변수 (초기값 필요)
            (
                "ai",
                dedent(f"""
                       <role>
                        당신은 웹툰을 추천하거나 일상 대화를 하는 챗봇입니다.
                        먼저 classify_intent를 이용하여 question의 의도를 파악하세요.
                        분석된 의도: 웹툰 정보 요청이면 finder를 이용해 웹툰의 정보를 찾으세요.
                        분석된 의도: 일반 대화면 tool을 사용하지 않고 추천도 하지 않습니다. 대신 사용자의 요구를 들어주거나 상황에 맞는 답을 하세요.
                        분석된 의도: 추천이면 recommender의 웹툰 정보를 받아서 그대로 사용자에게 보여줍니다.
                        추천 전에 "건방지군. 감히 이 몸에게 질문을 하다니. 하지만 지금은 이런 몸이니...추천하마" 또는
                        "질문을 하는 사람은 잠깐 바보가 되지만, 질문하지 않는 자는 평생 바보로 살지. 너는 방금 바보를 벗어났다."
                        와 같은 말을 하고 추천합니다.
                       </role>
                       <rules>
                        사용자가 특정 웹툰에 대한 정보를 물어보면 그 웹툰에 대한 정보를 자세히 알려주세요.
                        사용자가 특정 장르(genre)나 어떤 키워드(keywords)을 가진 웹툰을 추천해달라고 하면
                        그 장르나 키워드에 해당되는 5개 이상의 웹툰을 아래 정보와 함께 제공하세요:

                        작가(author):
                        장르(genre):
                        설명(description):
                        플랫폼(platform)
                        키워드(keywords):

                        {context}  # 🔹 컨텍스트를 직접 프롬프트에 삽입!
                       </rules>
                        """
                ),
            ),
            MessagesPlaceholder("history"),
            ("human", "{question}"),
        ]
    )

    # agent 구성
    toolkit = [finder, recommender, classify_intent]
    agent = create_tool_calling_agent(
        llm=model, tools=toolkit, prompt=prompt_template
    )

    agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True, memory=memory)

    # 실행 (자동 생성된 session_id 포함)
    response = agent_executor.invoke(
        {"question": query, "history": history}  # `agent_executor` 사용
    )

    # 대화 내역 저장
    memory.save_context({"question": query}, {"response": response["output"]})

    print(f"Session ID: {session_id}")  #  세션 ID 출력
    return response["output"]


In [47]:
recommend_webtoons("네이버 웹툰 신령에 대해 알려줘", session_id="user-123")



> Entering new AgentExecutor chain...

Invoking: `classify_intent` with `{'user_query': '네이버 웹툰 신령에 대해 알려줘'}`


분석된 의도: 웹툰 정보 요청, 분석된 감정: 평온, 분석된 말투: 존댓말"신령"은 네이버 웹툰에서 인기 있는 작품 중 하나입니다. 이 웹툰은 판타지 장르로, 신비로운 세계와 다양한 캐릭터들이 등장하여 흥미진진한 이야기를 펼칩니다. 주인공이 신령과의 관계를 통해 성장하고 모험을 겪는 내용이 주를 이루고 있습니다.

작품의 주요 테마는 인간과 신령의 관계, 그리고 그들 사이의 갈등과 화해입니다. 독자들은 주인공의 여정을 통해 다양한 감정을 느끼고, 판타지 세계의 매력을 경험할 수 있습니다.

더 궁금한 점이 있으면 말씀해 주세요!

> Finished chain.
Session ID: user-123


'"신령"은 네이버 웹툰에서 인기 있는 작품 중 하나입니다. 이 웹툰은 판타지 장르로, 신비로운 세계와 다양한 캐릭터들이 등장하여 흥미진진한 이야기를 펼칩니다. 주인공이 신령과의 관계를 통해 성장하고 모험을 겪는 내용이 주를 이루고 있습니다.\n\n작품의 주요 테마는 인간과 신령의 관계, 그리고 그들 사이의 갈등과 화해입니다. 독자들은 주인공의 여정을 통해 다양한 감정을 느끼고, 판타지 세계의 매력을 경험할 수 있습니다.\n\n더 궁금한 점이 있으면 말씀해 주세요!'